In [13]:
import numpy as np
import pandas as pd
import math


train = pd.read_csv('crime-train.txt',delimiter='\t')
test = pd.read_csv('crime-test.txt',delimiter='\t')
train.head()


,ViolentCrimesPerPop,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,0.67,-0.45,-1.85,-1.06,0.67,0.08,-0.85,-0.34,0.68,-0.24,...,-0.23,-0.02,-0.53,-1.08,-0.13,-0.66,-0.41,-0.56,1.26,-0.39
1,0.43,-0.45,-0.27,-0.22,-0.17,-0.34,-0.58,-0.50,-1.57,-0.29,...,-0.23,-0.33,-0.58,0.03,0.22,-0.46,-0.50,-0.11,-0.62,-0.39
2,0.12,-0.14,1.87,0.55,0.04,0.02,-1.19,-0.03,0.68,1.05,...,-0.23,-0.11,-1.51,1.07,0.07,-0.01,-0.41,0.77,0.52,-0.39
3,0.03,-0.38,0.53,-0.28,-0.79,-0.64,-0.35,-0.34,0.46,0.66,...,-0.23,-0.46,0.54,0.58,-0.08,-0.61,-0.23,-0.70,-0.62,-0.39
4,0.14,-0.30,-1.12,-0.74,-0.10,-0.40,-0.30,-0.19,0.68,0.76,...,-0.23,2.10,-0.92,-0.25,0.52,-0.06,-0.50,1.71,-0.27,-0.39


In [14]:
# store expected outcomes (1st column) into y_train; everything else for prediction goes into X_train
X_train = train.drop('ViolentCrimesPerPop',axis=1)
y_train = train['ViolentCrimesPerPop']

#convert values from string to float
X_train = np.float64(X_train)


ones = np.ones(len(X_train))
X_train = np.column_stack((X_train, ones))

y_train = np.float64(y_train)
# turn the array into a matrix (with one column)
y_train = np.reshape(y_train, (-1,1))

# check to make sure we have the correct amount of communities and features
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


# do the same thing from above but for the testing subset
X_test = test.drop('ViolentCrimesPerPop',axis=1)
y_test = test['ViolentCrimesPerPop']

X_test = np.float64(X_test)

ones = np.ones(len(X_test))
X_test = np.column_stack((X_test, ones))

y_test = np.float64(y_test)
# turn the array into a matrix (with one column)
y_test = np.reshape(y_test, (-1,1))

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1595, 96)
y_train shape: (1595, 1)
X_test shape: (399, 96)
y_test shape: (399, 1)


In [15]:
# this function simply computs the root mean square error (RMSE) by taking in two parameters: the predicted outcome matrix and the actual outcome matrix
def RMSE(prediction, actual):
    N = len(prediction)
    #some sorta issue here
    difference = prediction - actual
    total = 0
    
    for instance in difference:
        total += (instance ** 2)
        
    total_error = math.sqrt(total/N)
    return total_error

In [16]:
# perform ridge regression and return list of predicted outcomes that each correspond to their actual values
def ridge_regression(x, y, test, lambda_value):
    left = np.linalg.inv(np.dot(x.T, x) + (lambda_value * np.identity(len(x.T))))
    right = np.dot(x.T, y)
    w = np.dot(left, right)
    
    prediction = []
    
    for Xtest in test:
        prediction.append(np.dot(Xtest.T, w))
        
    return prediction


In [31]:
lambda_value = 400
RMSEs = np.zeros((10,5))
minimum = 0

# do p-fold cross validation with p = 5
for segment in range(len(RMSEs)):
    for p in range(5):
        # train with 4/5 of our dataset
        X_train_p = np.concatenate((X_train[0:int(p*(len(X_train)/5))], X_train[int((p+1)*(len(X_train)/5)):int(len(X_train))]))
        y_train_p = np.concatenate((y_train[0:int(p*(len(y_train)/5))], y_train[int((p+1)*(len(y_train)/5)):int(len(y_train))]))

        # use the remaining 1/5 dataset to validate our results
        X_validate_p = X_train[int(p*(len(X_train)/5)):int((p+1)*(len(X_train)/5))]
        y_validate_p = y_train[int(p*(len(y_train)/5)):int((p+1)*(len(y_train)/5))]

        # perform ridge regression on each fold and each lambda 
        prediction = ridge_regression(X_train_p, y_train_p, X_validate_p, lambda_value)

        # for each lambda, put all 5 of the different RMSEs on the same row
        RMSEs[segment, p] = RMSE(prediction, y_validate_p)
        
    lambda_value /= 2

print("RMSE values:\n",RMSEs, "\n")
    
# get average RMSE for each lambda and choose optimal lambda
for i in range(len(RMSEs)-1):
    if (sum(RMSEs[i+1]) / 5) < (sum(RMSEs[minimum]) / 5):
        minimum = i+1

for j in range(len(RMSEs)):
    print("𝜆 =", (400 / (2 ** j)), ", RMSE:", sum(RMSEs[j]) / 5)

optimal_lambda = 400 / (2 ** minimum)

print("\nThe optimal 𝜆 is:", optimal_lambda)
print("Training RMSE for ridge regression using closed form solution is", (sum(RMSEs[minimum]) / 5))

0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
0 ------- 319
319 ------- 638
638 ------- 957
957 ------- 1276
1276 ------- 1595
RMSE values:
 [[0.16031626 0.14530191 0.15755165 0.14669202 0.13770204]
 [0.1504947  0.13916403 0.14576515 0.13872299 0.12932324]
 [0.14641669 0.13761947 0.13961026 0.13589421 0.12684322]
 [

In [32]:
#step2

# compute loss function for linear regression
def linear_gradient_descent(x, y, w):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    min_w = np.dot(left, right)
    return min_w

alpha = 5*(10**-5)
# use gradient descent until convergence to obtain w
L1 = 1
L0 = 0
# generate a Gaussian (normal) distribution for our initial value, based off how many features we have
w0 = np.random.normal(0, 1, (X_train.shape[1],1))

while (abs(L1 - L0) > (10**-5)):
    w1 = w0 - (alpha * np.dot(X_train.T, (np.dot(X_train, w0) - y_train)))
    L0 = linear_gradient_descent(X_train, y_train, w0)
    L1 = linear_gradient_descent(X_train, y_train, w1)
    w0 = w1

In [33]:
# this function takes in an NxK array and returns predictions in the form of an Nx1 array
def problem2(w_value, matrix):
    prediction = []
    for x in matrix:
        prediction.append(np.dot(w_value.T, x))
    return prediction
train_prediction = []
test_prediction = []

# make predictions and calculate RMSE for training data
train_prediction = problem2(w0, X_train)

train_linear_RSME = RMSE(train_prediction, y_train)
print("Training RMSE for linear regression using gradient descent is", train_linear_RSME)

# make predictions and calculate RMSE for test data
test_prediction = problem2(w0, X_test)

test_linear_RMSE = RMSE(test_prediction, y_test)
print("Testing RMSE for linear regression using gradient descent is", test_linear_RMSE)

Training RMSE for linear regression using gradient descent is 0.12790755860294317
Testing RMSE for linear regression using gradient descent is 0.1463459470401774


In [34]:
#step3
# compute loss function for ridge regression
def ridge_gradient_descent(x, y, w, lambda_value):
    left = (y - np.dot(x, w)).T
    right = (y - np.dot(x, w))
    min_W = np.dot(left, right) + (optimal_lambda * np.dot(w0.T, w0))
    
    return min_W

alpha = 5*(10**-5)
# this function takes in an NxP array and returns predictions in the form of an Nx1 array
def problem3(w, validation):
    predictions = []
    for Xnew in validation:
        predictions.append(np.dot(w.T, Xnew))
    return predictions

In [35]:
# similar to problem 1, we need to find the optimal lambda
lambda_value = 400
RMSEs = np.zeros((10,5))
minimum = 0

# do p-fold cross validation with p = 5
for segment in range(len(RMSEs)):
    for p in range(5):
        # train with 4/5 of our dataset
        X_train_p = np.concatenate((X_train[0:int(p*(len(X_train)/5))], X_train[int((p+1)*(len(X_train)/5)):int(len(X_train))]))
        y_train_p = np.concatenate((y_train[0:int(p*(len(y_train)/5))], y_train[int((p+1)*(len(y_train)/5)):int(len(y_train))]))
        
        # use the remaining 1/5 dataset to validate our results
        X_validate_p = X_train[int(p*(len(X_train)/5)):int((p+1)*(len(X_train)/5))]
        y_validate_p = y_train[int(p*(len(y_train)/5)):int((p+1)*(len(y_train)/5))]
        
        # perform ridge regression using gradient descent on each fold and each lambda
        L1 = 1
        L0 = 0
        # generate a Gaussian (normal) distribution for our initial value, based off how many features we have
        w0 = np.random.normal(0, 1, (X_train.shape[1],1))
        while (abs(L1 - L0) > (10**-5)):
            w1 = w0 - (alpha * (np.dot(X_train_p.T, (np.dot(X_train_p, w0) - y_train_p)) + (lambda_value * w0)))
            L0 = ridge_gradient_descent(X_train_p, y_train_p, w0, lambda_value)
            L1 = ridge_gradient_descent(X_train_p, y_train_p, w1, lambda_value)
            w0 = w1
        
        validated_results = []
        validated_results = problem3(w0, X_validate_p)

        # for each lambda, put all 5 of the different RMSEs on the same row
        RMSEs[segment, p] = RMSE(validated_results, y_validate_p)
        
    lambda_value /= 2

print("RMSE values:\n",RMSEs, "\n")
    
# get average RMSE for each lambda and choose optimal lambda
for i in range(len(RMSEs)-1):
    if (sum(RMSEs[i+1]) / 5) < (sum(RMSEs[minimum]) / 5):
        minimum = i+1

for j in range(len(RMSEs)):
    print("𝜆 =", (400 / (2 ** j)), ", RMSE:", sum(RMSEs[j]) / 5)

optimal_lambda = 400 / (2 ** minimum)

print("\nThe optimal 𝜆 is:", optimal_lambda)
print("Training RMSE for ridge regression using gradient descent is", (sum(RMSEs[minimum]) / 5))

RMSE values:
 [[0.16036734 0.14530394 0.15755075 0.14669244 0.13769954]
 [0.15048019 0.13914529 0.14639289 0.13859329 0.12930741]
 [0.14628951 0.13761093 0.13956085 0.13577115 0.12686385]
 [0.14511315 0.13752131 0.13637895 0.13508807 0.12645488]
 [0.1450747  0.1383717  0.13462788 0.13467072 0.12790876]
 [0.14528744 0.13811813 0.13430071 0.13453009 0.1286557 ]
 [0.14554256 0.13836213 0.13298841 0.13373619 0.12988958]
 [0.14610823 0.14040525 0.13332692 0.13464729 0.13078923]
 [0.14626582 0.13891548 0.13369769 0.13380819 0.13171751]
 [0.14740543 0.14100676 0.13323627 0.13581039 0.13165056]] 

𝜆 = 400.0 , RMSE: 0.1495228013931734
𝜆 = 200.0 , RMSE: 0.14078381574446935
𝜆 = 100.0 , RMSE: 0.13721925701336785
𝜆 = 50.0 , RMSE: 0.1361112731835094
𝜆 = 25.0 , RMSE: 0.13613075274996334
𝜆 = 12.5 , RMSE: 0.1361784139237036
𝜆 = 6.25 , RMSE: 0.1361037737218622
𝜆 = 3.125 , RMSE: 0.13705538433921885
𝜆 = 1.5625 , RMSE: 0.1368809381080592
𝜆 = 0.78125 , RMSE: 0.13782188165850576

The optimal 𝜆 is: 6.25
Train

In [36]:
# re-compute w0 with the optimal lambda and all data from training subset so we can get results for our testing subset
L1 = 1
L0 = 0
# generate a Gaussian (normal) distribution for our initial value, based off how many features we have
w0 = np.random.normal(0, 1, (X_train.shape[1],1))


#this while loop m
while (abs(L1 - L0) > (10**-5)):
    w1 = w0 - (alpha * (np.dot(X_train.T, (np.dot(X_train, w0) - y_train)) + (optimal_lambda * w0)))
    L0 = ridge_gradient_descent(X_train, y_train, w0, lambda_value)
    L1 = ridge_gradient_descent(X_train, y_train, w1, lambda_value)
    w0 = w1

test_predictions = []
test_predictions = problem3(w0, X_test)

test_ridge_RMSE = RMSE(test_predictions, y_test)
print("Testing RMSE for ridge regression using gradient descent is", test_ridge_RMSE)

Testing RMSE for ridge regression using gradient descent is 0.1456727618821144
